In [1]:
import numpy as np
import random

In [2]:
# Define the grid size
GRID_SIZE = 4

# Define the initial position of the sheep
INITIAL_SHEEP1_POSITION = (3, 0)
INITIAL_SHEEP2_POSITION = (0, 3)

# Define the initial energy of the sheep
INITIAL_SHEEP1_ENERGY = 10000
INITIAL_SHEEP2_ENERGY = 10000

# Define the grass probability matrix
GRASS_MATRIX = np.zeros((GRID_SIZE, GRID_SIZE))

# Define the grass regrowth probability matrix
GRASS_REGROWTH_PROBABILITY_MATRIX = np.zeros((GRID_SIZE, GRID_SIZE))

In [3]:
GRASS_MATRIX [0][0]=1
GRASS_MATRIX [3][3]=1


GRASS_REGROWTH_PROBABILITY_MATRIX[0][0]=1
GRASS_REGROWTH_PROBABILITY_MATRIX[3][3]=1


In [4]:
GRASS_MATRIX

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [5]:
GRASS_REGROWTH_PROBABILITY_MATRIX

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [6]:
# Define the Q-Table
Q_TABLE_SHEEP1 = np.zeros((GRID_SIZE, GRID_SIZE, 5))
Q_TABLE_SHEEP2 = np.zeros((GRID_SIZE, GRID_SIZE, 5))

# Define the actions (up, right, down, left)
actions = ['up', 'right', 'down', 'left','stay']

# Define the learning rate
epsilon = 0.8

# Define the discount factor
discount_factor = 0.95

# Define the exploration rate
learning_rate = 0.3

# Initialize the sheep' position and energy
sheep1_position = INITIAL_SHEEP1_POSITION
sheep2_position = INITIAL_SHEEP2_POSITION
sheep1_energy = INITIAL_SHEEP1_ENERGY
sheep2_energy = INITIAL_SHEEP2_ENERGY

# Define the maximum number of steps for the sheep to take
MAX_STEPS = 10000

# Define a flag to keep track of whether the sheep has died
sheep_dead = False

In [7]:
def get_next_action_sheep1(current_row_index, current_column_index, learning_rate):
    # Define a list of valid actions based on the current position
    valid_actions = []
    valid_actions.append(4) #no matter where the sheep is, the sheep can always choose to stay at the same position
    if current_row_index > 0:
        valid_actions.append(0)  # up
    if current_row_index < GRID_SIZE - 1:
        valid_actions.append(2)  # down
    if current_column_index > 0:
        valid_actions.append(3)  # left
    if current_column_index < GRID_SIZE - 1:
        valid_actions.append(1)  # right
    # If a randomly chosen value between 0 and 1 is less than learning_rate, 
    # then choose a random valid action
    if np.random.random() < learning_rate:
        return np.random.choice(valid_actions)
    else: # Choose the most promising value from the Q-table for this state among valid actions
        valid_q_values = [Q_TABLE_SHEEP1[current_row_index, current_column_index, action_index] for action_index in valid_actions]
        max_q_value = np.max(valid_q_values)
        indices_with_max_q_value = [i for i, q_value in enumerate(valid_q_values) if q_value == max_q_value]
        if len(indices_with_max_q_value) == 1:
            return valid_actions[indices_with_max_q_value[0]]
        else:
            return np.random.choice([valid_actions[i] for i in indices_with_max_q_value])
        

In [8]:
def get_next_action_sheep2(current_row_index, current_column_index, learning_rate):
    # Define a list of valid actions based on the current position
    valid_actions = []
    valid_actions.append(4) #no matter where the sheep is, the sheep can always choose to stay at the same position
    if current_row_index > 0:
        valid_actions.append(0)  # up
    if current_row_index < GRID_SIZE - 1:
        valid_actions.append(2)  # down
    if current_column_index > 0:
        valid_actions.append(3)  # left
    if current_column_index < GRID_SIZE - 1:
        valid_actions.append(1)  # right
    # If a randomly chosen value between 0 and 1 is less than learning_rate, 
    # then choose a random valid action
    if np.random.random() < learning_rate:
        return np.random.choice(valid_actions)
    else: # Choose the most promising value from the Q-table for this state among valid actions
        valid_q_values = [Q_TABLE_SHEEP2[current_row_index, current_column_index, action_index] for action_index in valid_actions]
    max_q_value = np.max(valid_q_values)
    indices_with_max_q_value = [i for i, q_value in enumerate(valid_q_values) if q_value == max_q_value]
    if len(indices_with_max_q_value) == 1:
        return valid_actions[indices_with_max_q_value[0]]
    else:
        return np.random.choice([valid_actions[i] for i in indices_with_max_q_value])    
        
    

In [9]:
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    
    if action_index == 0:  # up
        new_row_index -= 1
    elif action_index == 1:  # right
        new_column_index += 1
    elif action_index == 2:  # down
        new_row_index += 1
    elif action_index == 3:  # left
        new_column_index -= 1
    elif action_index ==4:
        new_row_index = new_row_index
        new_column_index = new_column_index
    
    return new_row_index, new_column_index


In [10]:
# Define the main game loop

GRASS_CURRENT = GRASS_MATRIX.copy()
for step in range(MAX_STEPS):
    # Check if any of the sheep has died
    if sheep1_energy <= 0 or sheep2_energy <=0 :
        sheep_dead = True
        print('game over: one or all of the sheep die')
        break
    sheep1_action = get_next_action_sheep1(sheep1_position[0], sheep1_position[1], learning_rate)
    sheep2_action = get_next_action_sheep2(sheep2_position[0], sheep2_position[1], learning_rate)
    new_sheep1_position = get_next_location(sheep1_position[0], sheep1_position[1], sheep1_action)
    new_sheep2_position = get_next_location(sheep2_position[0], sheep2_position[1], sheep2_action)
    reward1 = 0  # Initialize reward1
    reward2 = 0  # Initialize reward2
    

    # Check if there is grass at the new position
    if GRASS_CURRENT[new_sheep1_position] == 1:
        
        # Check if sheep 2 is also at the same position        
        if new_sheep1_position == new_sheep2_position:
            # Randomly determine which sheep gets to eat the grass
            if random.random() < 0.5:
                # Sheep 1 eats the grass and gains energy
                sheep1_energy += 5
                # Sheep 2 lose, lose one energy
                sheep2_energy -= 1
                # Remove the grass from the grid
                GRASS_CURRENT[new_sheep1_position] = 0
                # Reward sheep 1 for finding grass
                reward1 = 5
                # Penalize sheep 2 for not getting the grass
                reward2 = -1
                
            else:
                # Sheep 2 eats the grass and gains energy
                sheep2_energy += 5
                # Sheep 1 lose, lose one energy
                sheep1_energy -= 1
                # Remove the grass from the grid
                GRASS_CURRENT[new_sheep1_position] = 0
                # Reward sheep 2 for finding grass
                reward2 = 5
                # Penalize sheep 1 for not getting the grass
                reward1 = -1
                
        else:
            # Sheep 1 eats the grass and gains energy
            sheep1_energy += 5
            # Remove the grass from the grid
            GRASS_CURRENT[new_sheep1_position] = 0
            # Reward sheep 1 for finding grass
            reward1 = 5
            
    else:#no grass
        sheep1_energy -= 1
        reward1 = -1
        
            
    if GRASS_CURRENT[new_sheep2_position] == 1:
        
        if new_sheep1_position == new_sheep2_position:
            sheep2_energy = sheep2_energy
            reward2 = reward2
        else:
            #sheep2 can eat the grass and gains energy, sheep1 is not at the same position
            sheep2_energy += 5
            #Remove the grass from the grid
            GRASS_CURRENT[new_sheep2_position] = 0
            #Reward sheep2 for finding grass
            reward2 = 5
    else:
        sheep2_energy -= 1
        reward2 = -1
        
    
   
                

        
    # Update the Q-learning table for sheep 1
    max_q_value_sheep1 = np.max(Q_TABLE_SHEEP1[new_sheep1_position])
    q_value_sheep1 = (1 - epsilon) * Q_TABLE_SHEEP1[sheep1_position][sheep1_action] + epsilon * (reward1 + discount_factor * max_q_value_sheep1)
    Q_TABLE_SHEEP1[sheep1_position][sheep1_action] = q_value_sheep1

    # Update the sheep's position for sheep 1
    sheep1_position = new_sheep1_position

    # Update the Q-learning table for sheep 2
    max_q_value_sheep2 = np.max(Q_TABLE_SHEEP2[new_sheep2_position])
    q_value_sheep2 = (1 - epsilon) * Q_TABLE_SHEEP2[sheep2_position][sheep2_action] + epsilon * (reward2 + discount_factor * max_q_value_sheep2)
    Q_TABLE_SHEEP2[sheep2_position][sheep2_action] = q_value_sheep2

    # Update the sheep's position for sheep 2
    sheep2_position = new_sheep2_position
    
  

    # Regrow the grass
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if GRASS_MATRIX[i][j] == 1 and GRASS_CURRENT[i][j] == 0 and random.uniform(0, 1) < GRASS_REGROWTH_PROBABILITY_MATRIX[i][j]:
                GRASS_CURRENT[i][j] = 1
    # Print the sheep positions
    print("Sheep1 : ", sheep1_position, sheep1_energy, "Sheep2 : ", sheep2_position, sheep2_energy)
    #print("Sheep2 position and energy: ", sheep2_position, sheep2_energy)   
    print
    

Sheep1 :  (3, 0) 9999 Sheep2 :  (1, 3) 9999
Sheep1 :  (3, 1) 9998 Sheep2 :  (2, 3) 9998
Sheep1 :  (3, 1) 9997 Sheep2 :  (2, 2) 9997
Sheep1 :  (3, 0) 9996 Sheep2 :  (2, 1) 9996
Sheep1 :  (2, 0) 9995 Sheep2 :  (3, 1) 9995
Sheep1 :  (2, 1) 9994 Sheep2 :  (2, 1) 9994
Sheep1 :  (3, 1) 9993 Sheep2 :  (2, 2) 9993
Sheep1 :  (3, 0) 9992 Sheep2 :  (1, 2) 9992
Sheep1 :  (2, 0) 9991 Sheep2 :  (1, 1) 9991
Sheep1 :  (3, 0) 9990 Sheep2 :  (2, 1) 9990
Sheep1 :  (3, 0) 9989 Sheep2 :  (2, 1) 9989
Sheep1 :  (3, 1) 9988 Sheep2 :  (2, 0) 9988
Sheep1 :  (3, 2) 9987 Sheep2 :  (3, 0) 9987
Sheep1 :  (3, 2) 9986 Sheep2 :  (2, 0) 9986
Sheep1 :  (3, 3) 9991 Sheep2 :  (1, 0) 9985
Sheep1 :  (3, 2) 9990 Sheep2 :  (0, 0) 9990
Sheep1 :  (3, 3) 9995 Sheep2 :  (0, 0) 9995
Sheep1 :  (3, 2) 9994 Sheep2 :  (0, 0) 10000
Sheep1 :  (3, 3) 9999 Sheep2 :  (0, 0) 10005
Sheep1 :  (3, 2) 9998 Sheep2 :  (0, 0) 10010
Sheep1 :  (3, 3) 10003 Sheep2 :  (1, 0) 10009
Sheep1 :  (3, 2) 10002 Sheep2 :  (0, 0) 10014
Sheep1 :  (3, 3) 10007 Sh

Sheep1 :  (3, 3) 11227 Sheep2 :  (0, 0) 11220
Sheep1 :  (3, 3) 11232 Sheep2 :  (0, 0) 11225
Sheep1 :  (3, 3) 11237 Sheep2 :  (0, 0) 11230
Sheep1 :  (3, 3) 11242 Sheep2 :  (0, 0) 11235
Sheep1 :  (3, 3) 11247 Sheep2 :  (0, 0) 11240
Sheep1 :  (3, 3) 11252 Sheep2 :  (0, 0) 11245
Sheep1 :  (3, 3) 11257 Sheep2 :  (0, 0) 11250
Sheep1 :  (3, 3) 11262 Sheep2 :  (0, 0) 11255
Sheep1 :  (3, 3) 11267 Sheep2 :  (0, 0) 11260
Sheep1 :  (3, 3) 11272 Sheep2 :  (0, 0) 11265
Sheep1 :  (3, 2) 11271 Sheep2 :  (0, 0) 11270
Sheep1 :  (3, 3) 11276 Sheep2 :  (0, 0) 11275
Sheep1 :  (3, 3) 11281 Sheep2 :  (0, 0) 11280
Sheep1 :  (3, 3) 11286 Sheep2 :  (0, 0) 11285
Sheep1 :  (3, 3) 11291 Sheep2 :  (0, 0) 11290
Sheep1 :  (3, 3) 11296 Sheep2 :  (0, 1) 11289
Sheep1 :  (3, 2) 11295 Sheep2 :  (0, 0) 11294
Sheep1 :  (3, 3) 11300 Sheep2 :  (0, 0) 11299
Sheep1 :  (3, 3) 11305 Sheep2 :  (0, 0) 11304
Sheep1 :  (3, 2) 11304 Sheep2 :  (0, 0) 11309
Sheep1 :  (3, 3) 11309 Sheep2 :  (0, 0) 11314
Sheep1 :  (3, 3) 11314 Sheep2 :  (

Sheep1 :  (3, 3) 12766 Sheep2 :  (1, 0) 12576
Sheep1 :  (3, 3) 12771 Sheep2 :  (0, 0) 12581
Sheep1 :  (3, 3) 12776 Sheep2 :  (0, 0) 12586
Sheep1 :  (3, 2) 12775 Sheep2 :  (0, 0) 12591
Sheep1 :  (3, 3) 12780 Sheep2 :  (0, 0) 12596
Sheep1 :  (3, 3) 12785 Sheep2 :  (0, 0) 12601
Sheep1 :  (3, 3) 12790 Sheep2 :  (0, 1) 12600
Sheep1 :  (3, 3) 12795 Sheep2 :  (0, 0) 12605
Sheep1 :  (3, 3) 12800 Sheep2 :  (0, 0) 12610
Sheep1 :  (2, 3) 12799 Sheep2 :  (0, 0) 12615
Sheep1 :  (2, 2) 12798 Sheep2 :  (0, 0) 12620
Sheep1 :  (3, 2) 12797 Sheep2 :  (0, 0) 12625
Sheep1 :  (3, 3) 12802 Sheep2 :  (0, 0) 12630
Sheep1 :  (3, 3) 12807 Sheep2 :  (0, 0) 12635
Sheep1 :  (3, 3) 12812 Sheep2 :  (1, 0) 12634
Sheep1 :  (3, 3) 12817 Sheep2 :  (2, 0) 12633
Sheep1 :  (3, 2) 12816 Sheep2 :  (2, 1) 12632
Sheep1 :  (3, 3) 12821 Sheep2 :  (1, 1) 12631
Sheep1 :  (3, 3) 12826 Sheep2 :  (0, 1) 12630
Sheep1 :  (3, 2) 12825 Sheep2 :  (0, 1) 12629
Sheep1 :  (3, 3) 12830 Sheep2 :  (0, 0) 12634
Sheep1 :  (3, 3) 12835 Sheep2 :  (

Sheep1 :  (3, 3) 14288 Sheep2 :  (1, 0) 14020
Sheep1 :  (3, 3) 14293 Sheep2 :  (1, 1) 14019
Sheep1 :  (3, 3) 14298 Sheep2 :  (0, 1) 14018
Sheep1 :  (3, 2) 14297 Sheep2 :  (0, 0) 14023
Sheep1 :  (3, 3) 14302 Sheep2 :  (0, 0) 14028
Sheep1 :  (3, 3) 14307 Sheep2 :  (0, 0) 14033
Sheep1 :  (3, 2) 14306 Sheep2 :  (0, 0) 14038
Sheep1 :  (3, 1) 14305 Sheep2 :  (0, 0) 14043
Sheep1 :  (3, 0) 14304 Sheep2 :  (0, 0) 14048
Sheep1 :  (3, 1) 14303 Sheep2 :  (0, 0) 14053
Sheep1 :  (3, 1) 14302 Sheep2 :  (0, 1) 14052
Sheep1 :  (2, 1) 14301 Sheep2 :  (0, 0) 14057
Sheep1 :  (2, 2) 14300 Sheep2 :  (0, 0) 14062
Sheep1 :  (3, 2) 14299 Sheep2 :  (0, 0) 14067
Sheep1 :  (3, 3) 14304 Sheep2 :  (0, 0) 14072
Sheep1 :  (3, 3) 14309 Sheep2 :  (1, 0) 14071
Sheep1 :  (3, 3) 14314 Sheep2 :  (0, 0) 14076
Sheep1 :  (3, 3) 14319 Sheep2 :  (0, 0) 14081
Sheep1 :  (3, 3) 14324 Sheep2 :  (0, 0) 14086
Sheep1 :  (3, 3) 14329 Sheep2 :  (0, 0) 14091
Sheep1 :  (3, 3) 14334 Sheep2 :  (0, 0) 14096
Sheep1 :  (3, 3) 14339 Sheep2 :  (

Sheep1 :  (3, 3) 15689 Sheep2 :  (0, 0) 15351
Sheep1 :  (2, 3) 15688 Sheep2 :  (0, 0) 15356
Sheep1 :  (3, 3) 15693 Sheep2 :  (0, 0) 15361
Sheep1 :  (3, 3) 15698 Sheep2 :  (0, 0) 15366
Sheep1 :  (3, 3) 15703 Sheep2 :  (0, 0) 15371
Sheep1 :  (3, 3) 15708 Sheep2 :  (0, 0) 15376
Sheep1 :  (3, 3) 15713 Sheep2 :  (0, 0) 15381
Sheep1 :  (3, 3) 15718 Sheep2 :  (1, 0) 15380
Sheep1 :  (3, 3) 15723 Sheep2 :  (0, 0) 15385
Sheep1 :  (3, 3) 15728 Sheep2 :  (0, 0) 15390
Sheep1 :  (3, 3) 15733 Sheep2 :  (1, 0) 15389
Sheep1 :  (3, 3) 15738 Sheep2 :  (2, 0) 15388
Sheep1 :  (3, 3) 15743 Sheep2 :  (2, 1) 15387
Sheep1 :  (3, 3) 15748 Sheep2 :  (2, 0) 15386
Sheep1 :  (3, 3) 15753 Sheep2 :  (2, 1) 15385
Sheep1 :  (3, 3) 15758 Sheep2 :  (1, 1) 15384
Sheep1 :  (3, 3) 15763 Sheep2 :  (2, 1) 15383
Sheep1 :  (3, 3) 15768 Sheep2 :  (2, 0) 15382
Sheep1 :  (3, 3) 15773 Sheep2 :  (2, 1) 15381
Sheep1 :  (3, 3) 15778 Sheep2 :  (1, 1) 15380
Sheep1 :  (3, 3) 15783 Sheep2 :  (0, 1) 15379
Sheep1 :  (3, 3) 15788 Sheep2 :  (

...

Sheep1 :  (3, 2) 39290 Sheep2 :  (0, 0) 38825
Sheep1 :  (3, 3) 39295 Sheep2 :  (0, 0) 38830
Sheep1 :  (3, 3) 39300 Sheep2 :  (0, 0) 38835
Sheep1 :  (3, 3) 39305 Sheep2 :  (0, 0) 38840
Sheep1 :  (3, 3) 39310 Sheep2 :  (0, 0) 38845
Sheep1 :  (3, 3) 39315 Sheep2 :  (0, 0) 38850
Sheep1 :  (3, 3) 39320 Sheep2 :  (1, 0) 38849
Sheep1 :  (3, 2) 39319 Sheep2 :  (0, 0) 38854
Sheep1 :  (3, 3) 39324 Sheep2 :  (0, 0) 38859
Sheep1 :  (3, 3) 39329 Sheep2 :  (0, 0) 38864
Sheep1 :  (3, 3) 39334 Sheep2 :  (1, 0) 38863
Sheep1 :  (2, 3) 39333 Sheep2 :  (0, 0) 38868
Sheep1 :  (3, 3) 39338 Sheep2 :  (0, 0) 38873
Sheep1 :  (3, 3) 39343 Sheep2 :  (0, 0) 38878
Sheep1 :  (3, 3) 39348 Sheep2 :  (0, 0) 38883
Sheep1 :  (3, 3) 39353 Sheep2 :  (0, 0) 38888
Sheep1 :  (3, 3) 39358 Sheep2 :  (0, 0) 38893
Sheep1 :  (3, 3) 39363 Sheep2 :  (0, 0) 38898
Sheep1 :  (3, 3) 39368 Sheep2 :  (0, 0) 38903
Sheep1 :  (3, 3) 39373 Sheep2 :  (0, 0) 38908
Sheep1 :  (3, 3) 39378 Sheep2 :  (0, 1) 38907
Sheep1 :  (3, 3) 39383 Sheep2 :  (

Sheep1 :  (2, 3) 40606 Sheep2 :  (1, 0) 40225
Sheep1 :  (3, 3) 40611 Sheep2 :  (0, 0) 40230
Sheep1 :  (3, 3) 40616 Sheep2 :  (0, 0) 40235
Sheep1 :  (3, 3) 40621 Sheep2 :  (0, 0) 40240
Sheep1 :  (3, 3) 40626 Sheep2 :  (0, 0) 40245
Sheep1 :  (3, 3) 40631 Sheep2 :  (0, 0) 40250
Sheep1 :  (3, 3) 40636 Sheep2 :  (1, 0) 40249
Sheep1 :  (2, 3) 40635 Sheep2 :  (0, 0) 40254
Sheep1 :  (2, 3) 40634 Sheep2 :  (0, 1) 40253
Sheep1 :  (3, 3) 40639 Sheep2 :  (0, 0) 40258
Sheep1 :  (3, 3) 40644 Sheep2 :  (0, 0) 40263
Sheep1 :  (3, 3) 40649 Sheep2 :  (0, 0) 40268
Sheep1 :  (2, 3) 40648 Sheep2 :  (0, 1) 40267
Sheep1 :  (3, 3) 40653 Sheep2 :  (0, 0) 40272
Sheep1 :  (3, 2) 40652 Sheep2 :  (0, 0) 40277
Sheep1 :  (3, 3) 40657 Sheep2 :  (1, 0) 40276
Sheep1 :  (3, 3) 40662 Sheep2 :  (0, 0) 40281
Sheep1 :  (2, 3) 40661 Sheep2 :  (1, 0) 40280
Sheep1 :  (3, 3) 40666 Sheep2 :  (0, 0) 40285
Sheep1 :  (3, 3) 40671 Sheep2 :  (0, 0) 40290
Sheep1 :  (2, 3) 40670 Sheep2 :  (0, 0) 40295
Sheep1 :  (2, 3) 40669 Sheep2 :  (

Sheep1 :  (3, 3) 42126 Sheep2 :  (0, 0) 41709
Sheep1 :  (3, 3) 42131 Sheep2 :  (0, 1) 41708
Sheep1 :  (3, 3) 42136 Sheep2 :  (0, 1) 41707
Sheep1 :  (3, 2) 42135 Sheep2 :  (0, 0) 41712
Sheep1 :  (2, 2) 42134 Sheep2 :  (1, 0) 41711
Sheep1 :  (2, 3) 42133 Sheep2 :  (0, 0) 41716
Sheep1 :  (3, 3) 42138 Sheep2 :  (0, 0) 41721
Sheep1 :  (3, 3) 42143 Sheep2 :  (0, 0) 41726
Sheep1 :  (3, 3) 42148 Sheep2 :  (0, 0) 41731
Sheep1 :  (3, 3) 42153 Sheep2 :  (0, 0) 41736
Sheep1 :  (3, 3) 42158 Sheep2 :  (0, 0) 41741
Sheep1 :  (3, 3) 42163 Sheep2 :  (0, 0) 41746
Sheep1 :  (3, 3) 42168 Sheep2 :  (0, 0) 41751
Sheep1 :  (3, 3) 42173 Sheep2 :  (0, 0) 41756
Sheep1 :  (3, 3) 42178 Sheep2 :  (0, 0) 41761
Sheep1 :  (3, 3) 42183 Sheep2 :  (1, 0) 41760
Sheep1 :  (3, 2) 42182 Sheep2 :  (1, 0) 41759
Sheep1 :  (3, 3) 42187 Sheep2 :  (0, 0) 41764
Sheep1 :  (3, 3) 42192 Sheep2 :  (0, 0) 41769
Sheep1 :  (3, 2) 42191 Sheep2 :  (0, 0) 41774
Sheep1 :  (3, 3) 42196 Sheep2 :  (0, 0) 41779
Sheep1 :  (3, 3) 42201 Sheep2 :  (

Sheep1 :  (3, 3) 43599 Sheep2 :  (0, 0) 43224
Sheep1 :  (3, 3) 43604 Sheep2 :  (1, 0) 43223
Sheep1 :  (3, 3) 43609 Sheep2 :  (1, 0) 43222
Sheep1 :  (3, 3) 43614 Sheep2 :  (0, 0) 43227
Sheep1 :  (3, 3) 43619 Sheep2 :  (0, 0) 43232
Sheep1 :  (3, 3) 43624 Sheep2 :  (0, 0) 43237
Sheep1 :  (3, 2) 43623 Sheep2 :  (0, 0) 43242
Sheep1 :  (3, 3) 43628 Sheep2 :  (1, 0) 43241
Sheep1 :  (3, 3) 43633 Sheep2 :  (2, 0) 43240
Sheep1 :  (3, 3) 43638 Sheep2 :  (3, 0) 43239
Sheep1 :  (3, 3) 43643 Sheep2 :  (2, 0) 43238
Sheep1 :  (3, 3) 43648 Sheep2 :  (1, 0) 43237
Sheep1 :  (3, 3) 43653 Sheep2 :  (1, 0) 43236
Sheep1 :  (3, 3) 43658 Sheep2 :  (0, 0) 43241
Sheep1 :  (3, 3) 43663 Sheep2 :  (1, 0) 43240
Sheep1 :  (3, 3) 43668 Sheep2 :  (1, 0) 43239
Sheep1 :  (3, 3) 43673 Sheep2 :  (0, 0) 43244
Sheep1 :  (2, 3) 43672 Sheep2 :  (0, 0) 43249
Sheep1 :  (2, 3) 43671 Sheep2 :  (0, 0) 43254
Sheep1 :  (3, 3) 43676 Sheep2 :  (1, 0) 43253
Sheep1 :  (3, 3) 43681 Sheep2 :  (1, 0) 43252
Sheep1 :  (3, 3) 43686 Sheep2 :  (

Sheep1 :  (3, 3) 45151 Sheep2 :  (0, 0) 44626
Sheep1 :  (3, 3) 45156 Sheep2 :  (0, 0) 44631
Sheep1 :  (3, 3) 45161 Sheep2 :  (0, 0) 44636
Sheep1 :  (2, 3) 45160 Sheep2 :  (0, 0) 44641
Sheep1 :  (1, 3) 45159 Sheep2 :  (0, 0) 44646
Sheep1 :  (2, 3) 45158 Sheep2 :  (0, 1) 44645
Sheep1 :  (2, 3) 45157 Sheep2 :  (0, 0) 44650
Sheep1 :  (3, 3) 45162 Sheep2 :  (1, 0) 44649
Sheep1 :  (3, 3) 45167 Sheep2 :  (0, 0) 44654
Sheep1 :  (3, 3) 45172 Sheep2 :  (0, 0) 44659
Sheep1 :  (3, 3) 45177 Sheep2 :  (0, 0) 44664
Sheep1 :  (3, 2) 45176 Sheep2 :  (0, 0) 44669
Sheep1 :  (3, 2) 45175 Sheep2 :  (0, 0) 44674
Sheep1 :  (3, 3) 45180 Sheep2 :  (0, 0) 44679
Sheep1 :  (3, 3) 45185 Sheep2 :  (0, 0) 44684
Sheep1 :  (2, 3) 45184 Sheep2 :  (0, 0) 44689
Sheep1 :  (3, 3) 45189 Sheep2 :  (0, 0) 44694
Sheep1 :  (3, 3) 45194 Sheep2 :  (0, 0) 44699
Sheep1 :  (3, 3) 45199 Sheep2 :  (0, 0) 44704
Sheep1 :  (3, 3) 45204 Sheep2 :  (1, 0) 44703
Sheep1 :  (3, 3) 45209 Sheep2 :  (0, 0) 44708
Sheep1 :  (3, 3) 45214 Sheep2 :  (

In [11]:
Q_TABLE_SHEEP1

array([[[  0.        ,  64.85261106,  69.27734252,   0.        ,
          73.63232707],
        [  0.        ,  -0.96      ,  31.08486215,  69.36898088,
          20.4826426 ],
        [  0.        ,  -0.8       ,  -0.96      ,  -0.8       ,
          -0.8       ],
        [  0.        ,   0.        ,  88.27147982,  -0.8       ,
          -0.96      ]],

       [[ 73.67082651,  26.87790488,   3.7263007 ,   0.        ,
          37.74577189],
        [ 38.30294167,  -0.8       ,  61.76182908,   0.        ,
           0.        ],
        [ -0.8       ,  -0.8       ,  88.29995438,  -0.96      ,
          -0.96      ],
        [ 82.31734132,   0.        ,  94.        ,  82.20790653,
          88.29958404]],

       [[ 60.95438604,  -0.8       ,  -0.8       ,   0.        ,
           0.        ],
        [  5.60861724,  88.29997901,  70.48      ,   4.82356085,
          79.55954798],
        [ 82.7674665 ,  93.99999807,  94.        ,  82.74791644,
          88.29999999],
        [ 88.3   

In [12]:
Q_TABLE_SHEEP2

array([[[  0.        ,  94.        ,  94.        ,   0.        ,
         100.        ],
        [  0.        ,  88.3       ,  88.3       , 100.        ,
          94.        ],
        [  0.        ,  70.12023292,  82.87888186,  94.        ,
          88.09831224],
        [  0.        ,   0.        ,  77.37720029,  -0.992     ,
          -0.96      ]],

       [[100.        ,  88.3       ,  88.3       ,   0.        ,
          94.        ],
        [ 94.        ,  82.88472612,  82.88492315,  93.99923464,
          88.15871997],
        [ 70.48      ,  -0.8       ,  -0.96      ,  88.3       ,
          66.14799286],
        [ -0.8       ,   0.        ,  -0.8       ,  82.86448043,
          -0.8       ]],

       [[ 94.        ,  82.68101175,  82.55443728,   0.        ,
          86.43597662],
        [ 88.29999895,  65.87207448,  48.4916962 ,  85.44811467,
          77.92001812],
        [ -0.96      ,   2.24      ,  -0.8       ,  81.25788602,
           0.        ],
        [  0.    